# Notebook to verify the validation accuracy.

In [4]:
import string
import sys
import time
import matplotlib.pyplot as plt
import IPython.display as ipd
import argparse
import math
from tqdm import tqdm 
import random

import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats as st

## Scikit related
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, chi2, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDClassifier
from sklearn import datasets
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from scipy import signal
from scipy import integrate
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, KernelPCA
from sklearn.model_selection import cross_val_score
from scipy.stats import skew
from scipy.stats import norm, kurtosis
from sklearn.manifold import TSNE
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import mean_squared_error
from torch.autograd import Variable
# import xgboost as xgb
from sklearn.neighbors import KernelDensity

## Torch related
## Pytorch related
import torch
from torch._C import dtype
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from datetime import date

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device available is', device)

# Update the path to the data files
x_train_acc    = np.load('saved_var/x_Train_acc_30_0_3.npy', allow_pickle=True).astype(np.float32)
x_train_gyr    = np.load('saved_var/x_Train_gyr_30_0_3.npy', allow_pickle=True).astype(np.float32)
x_train_hr     = np.load('saved_var/x_Train_hr_30_0_3.npy', allow_pickle=True).astype(np.float32)
x_train_static = np.load('saved_var/x_Train_static_30_0_3.npy', allow_pickle=True).astype(np.float32)
y_train        = np.load('saved_var/y_Train_30_0_3.npy', allow_pickle=True).astype(np.float32)
y_train        = y_train[:,0]

print('Shape of x_train_acc', x_train_acc.shape)
print('Shape of x_train_gyro', x_train_gyr.shape)
print('Shape of x_train_hr', x_train_hr.shape)
print('Shape of x_train_static', x_train_static.shape)
print('Shape of y_train', y_train.shape)

Device available is cuda
Shape of x_train_acc (50567, 120, 3)
Shape of x_train_gyro (50567, 120, 3)
Shape of x_train_hr (50567, 120, 3)
Shape of x_train_static (50567, 120, 11)
Shape of y_train (50567, 1)


In [3]:
hidden_size = 64 
hidden_size_fc = 64
hidden_size_fc_2 = 64
learning_rate = 1e-3
num_layers = 1
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, hidden_size_fc, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        self.conv_acc = nn.Sequential(nn.Conv1d(input_size[0], 8, 3, padding='valid'),
                            nn.ReLU(),
                            nn.Conv1d(8, 8, 3, padding='valid'),
                            nn.ReLU(),
                            nn.Conv1d(8, 16, 3, padding='valid'),
                            nn.ReLU(),            
                           )  
        
        self.conv_gyr = nn.Sequential(nn.Conv1d(input_size[1], 8, 3, padding='valid'),
                            nn.ReLU(),
                            nn.Conv1d(8, 8, 3, padding='valid'),
                            nn.ReLU(),
                            nn.Conv1d(8, 16, 3, padding='valid'),
                            nn.ReLU(),            
                           )   
        
        self.conv_hr = nn.Sequential(nn.Conv1d(input_size[2], 8, 3, padding='valid'),
                            nn.ReLU(),
                            nn.Conv1d(8, 8, 3, padding='valid'),
                            nn.ReLU(),
                            nn.Conv1d(8, 16, 3, padding='valid'),
                            nn.ReLU(),            
                           )
                
        self.lstm = nn.LSTM(16*3,hidden_size,num_layers,batch_first=True)

        self.nn = nn.Sequential(nn.Linear(hidden_size+input_size[3], hidden_size_fc),
                            nn.ReLU(),
                            nn.Linear(hidden_size_fc, hidden_size_fc_2),
                            nn.ReLU(),
                            nn.Linear(hidden_size_fc_2, num_classes)       
                           )         

    
    def forward(self, x_acc, x_gyr, x_hr, x_static):
        
        # conv layers
        # print(x_acc.shape)
        x_acc = torch.permute(x_acc,(0,2,1))      
        x_gyr = torch.permute(x_gyr,(0,2,1))         
        x_hr = torch.permute(x_hr,(0,2,1))
        out_acc = self.conv_acc(x_acc)
        out_gyr = self.conv_gyr(x_gyr)
        out_hr = self.conv_hr(x_hr)
        out_acc = torch.permute(out_acc,(0,2,1))      
        out_gyr = torch.permute(out_gyr,(0,2,1))         
        out_hr = torch.permute(out_hr,(0,2,1))
        
        # concatenate all sensors except static
        out = torch.cat((out_acc, out_gyr, out_hr), 2)
        
        # LSTM        
        h0 = Variable(torch.zeros(self.num_layers, out.size(0), hidden_size).to(device))
        c0 = Variable(torch.zeros(self.num_layers, out.size(0), hidden_size).to(device))
        out, _ = self.lstm(out,(h0,c0))        
        # out = out[:, -1, :]
        out = torch.mean(out, dim=1)
        
        # concatenate with static features
        out = torch.cat((out, x_static[:,0,:]), 1)
        
        # nn
        out = self.nn(out)
        
        return out

    
input_size = [x_train_acc.shape[2], x_train_gyr.shape[2], x_train_hr.shape[2], x_train_static.shape[2]]
num_classes = 46
model = LSTM(input_size, hidden_size, hidden_size_fc, num_layers, num_classes).to(device)
model = torch.load('saved_var/epoch_2000.pth')
print(model)

LSTM(
  (conv_acc): Sequential(
    (0): Conv1d(3, 8, kernel_size=(3,), stride=(1,), padding=valid)
    (1): ReLU()
    (2): Conv1d(8, 8, kernel_size=(3,), stride=(1,), padding=valid)
    (3): ReLU()
    (4): Conv1d(8, 16, kernel_size=(3,), stride=(1,), padding=valid)
    (5): ReLU()
  )
  (conv_gyr): Sequential(
    (0): Conv1d(3, 8, kernel_size=(3,), stride=(1,), padding=valid)
    (1): ReLU()
    (2): Conv1d(8, 8, kernel_size=(3,), stride=(1,), padding=valid)
    (3): ReLU()
    (4): Conv1d(8, 16, kernel_size=(3,), stride=(1,), padding=valid)
    (5): ReLU()
  )
  (conv_hr): Sequential(
    (0): Conv1d(3, 8, kernel_size=(3,), stride=(1,), padding=valid)
    (1): ReLU()
    (2): Conv1d(8, 8, kernel_size=(3,), stride=(1,), padding=valid)
    (3): ReLU()
    (4): Conv1d(8, 16, kernel_size=(3,), stride=(1,), padding=valid)
    (5): ReLU()
  )
  (lstm): LSTM(48, 64, batch_first=True)
  (nn): Sequential(
    (0): Linear(in_features=75, out_features=64, bias=True)
    (1): ReLU()
    (2): 

In [14]:
folder_path = 'saved_var/valid_data_1h/' # for valid per day data

with torch.no_grad():
        actual_user_id = np.zeros((10000,))
        predicted_user_id = np.zeros((10000,))        

        for count, path in enumerate(os.listdir(folder_path)):
            user_id = path.split('_')[3].split('.')[0]
            # print(count,user_id)
            x_valid = np.load(folder_path+path, allow_pickle=True).astype(np.float32)
            x_valid = torch.from_numpy(x_valid).to(torch.float32).to(device)
            x_val_acc = x_valid[:,:,[0,1,2]]
            x_val_gyr = x_valid[:,:,[3,4,5]]
            x_val_hr = x_valid[:,:,[6,7,19]]
            x_val_static = x_valid[:,:,8:19]
            outputs = model(x_val_acc, x_val_gyr, x_val_hr, x_val_static)
            outputs = nn.Softmax(dim=1)(outputs)
            outputs = torch.argmax(outputs, dim=1)
            outputs = outputs.to('cpu')
            # print(st.mode(outputs)[0].item())
            actual_user_id[count,] = user_id
            predicted_user_id[count,] = st.mode(outputs)[0].item()
            
        actual_user_id = actual_user_id[0:count+1,]
        predicted_user_id = predicted_user_id[0:count+1,]
        per_day_val_acc = balanced_accuracy_score(actual_user_id, predicted_user_id)

In [15]:
per_day_val_acc

0.9003111019415367